In [2]:
!pip install pyodbc
!pip install pyspark
from pyspark import HiveContext
from pyspark.sql.types import *
from pyspark.sql import Row, functions as F
from pyspark.sql.window import Window

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845525 sha256=c4e5c1d3cd22c2daa981a6856ee01a5a75d7c02f1419acc4768f80ee11a81bc5
  Stored in directory: c:\users\cgdkobi\appdata\local\pip\cache\wheels\51\c8\18\298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [3]:
import pyodbc
import pandas as pd

In [4]:
conn=pyodbc.connect(
"Driver={SQL Server Native Client 11.0};"
"Server= CBTSQLR01;"
"Database=Tradesight;"
"Trusted_Connection=yes;") 

In [5]:
Lat_data = pd.read_sql("select *from Tradesight.dbo.ZipCode_Latitude_Longtitude", conn)
Lat_data  

C:\Program Files\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ZIP,LAT,LNG
0,00601,18.180555,-66.749961
1,00602,18.361945,-67.175597
2,00603,18.455183,-67.119887
3,00606,18.158345,-66.932911
4,00610,18.295366,-67.125135
...,...,...,...
33139,99923,56.002315,-130.041026
33140,99925,55.550204,-132.945933
33141,99926,55.138352,-131.470424
33142,99927,56.239062,-133.457924


In [6]:
Lat_data = Lat_data.rename(columns={"ZIP": "DEALER_ZIP"})
Lat_data

,DEALER_ZIP,LAT,LNG
0,00601,18.180555,-66.749961
1,00602,18.361945,-67.175597
2,00603,18.455183,-67.119887
3,00606,18.158345,-66.932911
4,00610,18.295366,-67.125135
...,...,...,...
33139,99923,56.002315,-130.041026
33140,99925,55.550204,-132.945933
33141,99926,55.138352,-131.470424
33142,99927,56.239062,-133.457924


In [ ]:
# determining the name of the file
file_name = 'Lat_Data.xlsx'
  
# saving the excel
Lat_data.to_excel(file_name)

## Exported Tradesight_Detail, 
### grouped by dealeraddress, ordered desc by vantage, taken the first unique record out it

In [31]:
# Vantage column in in Tradesight_Detail
data_cleaned = pd.read_sql("SELECT [VANTAGE], [DEALERNAME], [DEALERADDRESS], [DEALERCITY], [DEALERSTATE], [DEALERZIP], [LoanLeaseIndicator] FROM [Tradesight].[dbo].[Tradesight_Detail] where [VANTAGE] >= 420 and [VANTAGE] <= 640 and [DEALERNAME] is not null and [DEALERADDRESS] is not null and [DEALERCITY] is not null AND [LoanLeaseIndicator] in (0,5);",conn)
data_cleaned

C:\Program Files\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,VANTAGE,DEALERNAME,DEALERADDRESS,DEALERCITY,DEALERSTATE,DEALERZIP,LoanLeaseIndicator
0,487,CASCADE AUTOCENTER,148 EASY ST,WENATCHEE,WA,98801,5
1,554,CASCADE AUTOCENTER,148 EASY ST,WENATCHEE,WA,98801,5
2,553,LP EVANS MOTORS INC,3345 SW 8TH ST,MIAMI,FL,33135,5
3,634,LP EVANS MOTORS INC,3345 SW 8TH ST,MIAMI,FL,33135,5
4,615,LP EVANS MOTORS INC,3345 SW 8TH ST,MIAMI,FL,33135,5
...,...,...,...,...,...,...,...
9481290,580,FULLERTONAUTOMOTIVE CORP,RR 22E BOX,SOMERVILLE,NJ,08876,5
9481291,531,FULLERTONAUTOMOTIVE CORP,RR 22E BOX,SOMERVILLE,NJ,08876,5
9481292,595,TED BRITT FORD LINCOLN,SMART WHEELS OF FAIRFAX,FAIRFAX,VA,22030,5
9481293,558,TED BRITT FORD LINCOLN,SMART WHEELS OF FAIRFAX,FAIRFAX,VA,22030,5


In [32]:
data_cleaned.shape

(9481295, 7)

In [33]:
# main code

data_cleaned['ROW_NUM'] = data_cleaned.sort_values(['VANTAGE'], ascending=[False]) \
             .groupby(['DEALERADDRESS']) \
             .cumcount() + 1
print(data_cleaned)

         VANTAGE                DEALERNAME            DEALERADDRESS  \
0            487        CASCADE AUTOCENTER              148 EASY ST   
1            554        CASCADE AUTOCENTER              148 EASY ST   
2            553       LP EVANS MOTORS INC           3345 SW 8TH ST   
3            634       LP EVANS MOTORS INC           3345 SW 8TH ST   
4            615       LP EVANS MOTORS INC           3345 SW 8TH ST   
...          ...                       ...                      ...   
9481290      580  FULLERTONAUTOMOTIVE CORP               RR 22E BOX   
9481291      531  FULLERTONAUTOMOTIVE CORP               RR 22E BOX   
9481292      595    TED BRITT FORD LINCOLN  SMART WHEELS OF FAIRFAX   
9481293      558    TED BRITT FORD LINCOLN  SMART WHEELS OF FAIRFAX   
9481294      619           OCT PARTNERSHIP        ST RR 37E AND BOX   

         DEALERCITY DEALERSTATE DEALERZIP  LoanLeaseIndicator  ROW_NUM  
0         WENATCHEE          WA     98801                   5      320  
1

In [34]:
# for validating 
df = data_cleaned.sort_values(by=['DEALERADDRESS','ROW_NUM'])
df

,VANTAGE,DEALERNAME,DEALERADDRESS,DEALERCITY,DEALERSTATE,DEALERZIP,LoanLeaseIndicator,ROW_NUM
7163931,625,LAKELAND CAR COMPANY LLC,0-141 LAKE MICHIGAN DR,GRAND RAPIDS,MI,49534,0,1
7163930,618,LAKELAND CAR COMPANY LLC,0-141 LAKE MICHIGAN DR,GRAND RAPIDS,MI,49534,5,2
6900250,597,LAKELAND CAR COMPANY LLC,0-141 LAKE MICHIGAN DR,GRAND RAPIDS,MI,49534,5,3
4280269,595,LAKELAND CAR COMPANY LLC,0-141 LAKE MICHIGAN DR,GRAND RAPIDS,MI,49534,5,4
676988,589,LAKELAND CAR COMPANY LLC,0-141 LAKE MICHIGAN DR,GRAND RAPIDS,MI,49534,5,5
...,...,...,...,...,...,...,...,...
4760067,527,AUTO SOURCE OF WV INC,WEST,KINGWOOD,WV,26537,5,86
8104539,516,AUTO SOURCE OF WV INC,WEST,KINGWOOD,WV,26537,5,87
5560062,513,AUTO SOURCE OF WV INC,WEST,KINGWOOD,WV,26537,5,88
6724829,502,AUTO SOURCE OF WV INC,WEST,KINGWOOD,WV,26537,5,89


In [36]:
# taking where row_num = 1
df = df.loc[df['ROW_NUM']==1]
df

,VANTAGE,DEALERNAME,DEALERADDRESS,DEALERCITY,DEALERSTATE,DEALERZIP,LoanLeaseIndicator,ROW_NUM
7163931,625,LAKELAND CAR COMPANY LLC,0-141 LAKE MICHIGAN DR,GRAND RAPIDS,MI,49534,0,1
8370751,639,JAGUAR LAND ROVER FAIRFIELD,1 AND COMMERCE DR # 5,FAIRFIELD,CT,06825,5,1
1493982,640,TUSTIN BUICK GMC,1 AUTO CENTER DR,TUSTIN,CA,92782,5,1
1181474,640,STEVES CHEVROLET OF CHOWCHILLALLC,1 AUTO PARK DR,CHOWCHILLA,CA,93610,5,1
3190436,640,HONDA WORLD,1 AUTOCENTER DR,LOUISVILLE,KY,40299,5,1
...,...,...,...,...,...,...,...,...
8448117,640,MEISTER IMPORT MOTORS INC,W6995 CLEARY CT,GREENVILLE,WI,54942,5,1
5614807,640,SUMMIT FORD LLC,W8356 HOWARD DR,BEAVER DAM,WI,53916,5,1
7856117,638,B AND B AUTO INC,W8480 530TH AVE,ELLSWORTH,WI,54011,5,1
5392170,621,MACKINAC SALES INC,W858 US HIGHWAY 2 W,SAINT IGNACE,MI,49781,5,1


## combine detail, summary data, and lattitude data

In [37]:
df = df.rename(columns={"DEALERNAME": "DEALER_NAME", "DEALERADDRESS": "DEALER_ADDRESS",
                  "DEALERCITY": "DEALER_CITY", "DEALERSTATE": "DEALER_STATE",
                  "DEALERZIP": "DEALER_ZIP"})

In [39]:
df['DEALER_NAME'] = df['DEALER_NAME'].str.replace('#','')
df['DEALER_NAME'] = df['DEALER_NAME'].str.replace('.','')
df['DEALER_NAME'] = df['DEALER_NAME'].str.replace(', ','')
df['DEALER_ADDRESS'] = df['DEALER_ADDRESS'].str.replace('#','')
df['DEALER_ADDRESS'] = df['DEALER_ADDRESS'].str.replace('  ',' ')

C:\Users\cgdkobi\AppData\Local\Temp\ipykernel_13228\269347027.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['DEALER_NAME'] = df['DEALER_NAME'].str.replace('.','')


In [40]:
df.shape

(27898, 8)

In [41]:
mergedRes = pd.merge(Lat_data, df, on ='DEALER_ZIP')
mergedRes.shape

(27649, 10)

In [42]:
mergedRes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27649 entries, 0 to 27648
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DEALER_ZIP          27649 non-null  object 
 1   LAT                 27649 non-null  float64
 2   LNG                 27649 non-null  float64
 3   VANTAGE             27649 non-null  int64  
 4   DEALER_NAME         27649 non-null  object 
 5   DEALER_ADDRESS      27649 non-null  object 
 6   DEALER_CITY         27649 non-null  object 
 7   DEALER_STATE        27649 non-null  object 
 8   LoanLeaseIndicator  27649 non-null  int64  
 9   ROW_NUM             27649 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.3+ MB


In [43]:
mergedRes.shape

(27649, 10)

In [44]:
# determining the name of the file
file_name = 'MergedRes.xlsx'
  
# saving the excel
mergedRes.to_excel(file_name)

In [45]:
mergedRes.to_csv('C:/Users/cgdkobi/Downloads/MergedRes.csv')

In [47]:
mseaddress = pd.read_sql("SELECT * FROM [Tradesight].[dbo].[MSEAddresses];",conn)
mseaddress

,MSE,Territory,MobilePhone,Email,Address,City,State,Zip,UserID,Latitude,Longitude
0,Geoff Benedict,NSE,918-325-0236,gabenedict@cbtno.com,3350 Boyington Dr,Carrollton,TX,75006,gabenedict,32.964360,-96.842140
1,Mike Braaten,RSE,405-589-2191,mwbraaten@cbtno.com,123 S Meadowbrook Drive,Pottsboro,TX,75076,mwbraaten,26.715844,-80.070249
2,Brian Seymour,Jacksonville,904-476-3077,bfseymour@cbtno.com,150 Shores Blvd,St Augustine,FL,32086,bfseymour,29.766738,-81.306370
3,Kim Pokorny,Tampa,910-442-7334,kmpokorny@cbtno.com,13040 Tampa Oaks Blvd Unit 1311,Temple Terrace,FL,33637,kmpokorny,28.039845,-82.367516
4,David Schelnick,San Antonio,830-556-3794,dbschelnick@cbtno.com,1051 Country Club Dr,Seguin,TX,78155,dbschelnick,29.542743,-97.937075
5,Brandon Armelin,Houston,8326297114,bjarmelin@cbtno.com,4227 Duneberry Trail,Humble,TX,77346,bjarmelin,29.982212,-95.177644
6,Tim McBride,Ft. Worth,4697012394,trmcbride@cbtno.com,6101 Moor Hen Drive,Denton,TX,76208,trmcbride,33.212410,-97.072041
7,Roxanne Allen,Nashville,615-584-4735,rxallen@cbtno.com,413 Knob Ct,Franklin,TN,37064,rxallen,35.884862,-86.954138
8,Joshua Crawford,Houston,832-544-6520,jpcrawford@cbtno.com,1414 Evermore Manor Ln,Houston,TX,77073,jpcrawford,29.999166,-95.400119
9,Jerome Whitney,Baton Rouge,504-256-1494,jcwhitney@cbtno.com,2034 Chippewa St,New Orleans,LA,70130,jcwhitney,29.937353,-90.069648


In [53]:
# determining the name of the file
file_name = 'mseaddress.xlsx'
  
# saving the excel
mseaddress.to_excel(file_name)

In [54]:
mseaddress.to_csv('C:/Users/cgdkobi/Downloads/mseaddress.csv')